<a href="https://colab.research.google.com/github/ykim879/soccer.webscrapping/blob/Will's-Branch/Soccer_webscrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import requests
from bs4 import BeautifulSoup
import re
import threading
import queue
baseURL = 'https://www.soccerbase.com'
lock = threading.Lock()

In [14]:
#Decorator hopefully
import time
def timer(func):
  def timeIt(*args, **kwargs):
    start = time.time()
    func(*args, **kwargs)
    lock.acquire()
    print(func.__name__, 'took', end = ' ')
    print(f'{time.time() - start:0.2f}', 'seconds to run.')
    lock.release()
  return timeIt

In [15]:
@timer
def getEveryCompetitionData():
  request = requests.get("https://www.soccerbase.com/players/home.sd")
  soup = BeautifulSoup(request.content, "lxml")
  testing = soup.find('div', id = 'players-menu')
  for div in testing.find_all('div', class_ = 'soccerColumn'):
    if (re.search('Competition', div.text) != None):
      comp = div
  threads = []
  compQ = queue.Queue()
  for competition in comp.find_all('a'):
    t = threading.Thread(target = readCompetitionPage, args = (competition.text, baseURL + competition['href'], compQ))
    threads.append(t)
    t.start()
  for t in threads:
    t.join()
  counter = 1
  while not compQ.empty():
    print(counter, compQ.get()[0])
    counter += 1

In [16]:
#From player page for a team
def getTeamData(team, url, q):
  names = [team]
  request = requests.get(url)
  soup = BeautifulSoup(request.content, "lxml")
  test = soup.find('ul', attrs={'class':'infoList infoListBullet infoListLeftPadding infoListRightPadding'})
  for elem in test.find_all('b'):
    name = elem.find('a')
    names.append(name.text)
  #return names
  q.put(names)

#From competition page of teams
## seasons: dataframe consisted of each season value with the season date

In [17]:
import threading
import queue
@timer
def readCompetitionPage(competition, compURL, compQ):
  request = requests.get(compURL)
  soup = BeautifulSoup(request.content, "lxml")
  seasons = tournament_season_selector(soup)
  print(seasons)
  teamNames = soup.find('table', class_ = 'table')
  teams = {}
  threads = []
  q = queue.Queue()
  try:
    for url in teamNames.find_all('a'):
      t = threading.Thread(target = getTeamData, args = (url.text, baseURL + url['href'], q))
      threads.append(t)
      t.start()
    for t in threads:
      t.join()
    while not q.empty():
      teamData = q.get()
      teams[teamData[0]] = teamData[1:]

    compQ.put((competition, teams))
  except AttributeError:
    print(competition, 'failed to be read')

In [21]:
#readCompetitionPage('https://www.soccerbase.com/tournaments/tournament.sd?comp_id=20')
getEveryCompetitionData()

   value     date
0   1778  2020/21
1   1717  2019/20
2   1638  2018/19
3   1573  2017/18
4   1508  2016/17
5   1444  2015/16
6   1392  2014/15
7   1310  2013/14
8   1165  2012/13
9    936  2011/12
10    62  2010/11
11    61  2009/10
12   901  2008/09
13   902  2007/08
14   903  2006/07
15   904  2005/06
16   905  2004/05
17   906  2003/04
18   907  2002/03
19   908  2001/02
20   909  2000/01
21   910  1999/00
22   911  1998/99
   value     date
0   1780  2020/21
1   1708  2019/20
2   1633  2018/19
3   1594  2017/18
4   1525  2016/17
5   1449  2015/16
6   1369  2014/15
7   1319  2013/14
8   1183  2012/13
9   1004  2011/12
10    22  2010/11
11    23  2009/10
12   289  2008/09
13   290  2007/08
14   291  2006/07
15   292  2005/06
16   293  2004/05
17   294  2003/04
   value     date
0   1797  2020/21
1   1710  2019/20
2   1635  2018/19
3   1566  2017/18
4   1501  2016/17
5   1435  2015/16
6   1386  2014/15
7   1304  2013/14
8   1159  2012/13
9    939  2011/12
10     1  2010/11
11     2  

In [20]:
import re
import pandas as pd
def tournament_season_selector(soup):
  df = pd.DataFrame(columns = ["value", "date"])
  list = soup.find("select", id = "seasonSelect")
  for data in list.find_all("option")[1:]:
    df = df.append({"value": data["value"], "date" : data.text}, ignore_index = True)
  return df
